## **FASE IV: MODELADO**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sbn
from seaborn import kdeplot
from seaborn import lmplot
from seaborn import boxplot

import warnings #Advertencias
warnings.filterwarnings('ignore')

#Mostrar todas las columnas
pd.options.display.max_columns = 70
pd.options.display.max_rows = 70
#Ajuste de formato para evitar notacion científica
pd.options.display.float_format = '{:20,.5f}'.format

path = 'monopoly_cleaned_parquet'
monopoly = pd.read_parquet(path)

### **Getcho model right below here ya goddamn drongo**

In [ ]:
# こっちよ

